# Installations

In [ ]:
!pip install  datasets transformers

In [ ]:
!pip install --upgrade git+https://github.com/ramsrigouthamg/Questgen.ai

In [ ]:
!pip install --quiet git+https://github.com/boudinfl/pke.git@69337af9f9e72a25af6d7991eaa9869f1322dd72


In [ ]:
!python -m nltk.downloader universal_tagset
!python -m spacy download en

# Download Sense2vec wordvectors for generation of multiple choices

In [ ]:
!wget https://github.com/explosion/sense2vec/releases/download/v1.0.0/s2v_reddit_2015_md.tar.gz

In [ ]:
!tar -xvf  s2v_reddit_2015_md.tar.gz

In [ ]:
!ls s2v_old

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
import datasets

# Generate Questions

In [ ]:
from pprint import pprint
from Questgen import main

## Example

In [ ]:
payload = {
"input_text":"A more elaborate form of urban AC is the rhythmic oldies format, which focuses primarily on \"old school\" R&B and soul hits from the 1960s to the 1990s, including Motown and disco hits. The format includes soul or disco artists such as ABBA, The Village People, The Jackson 5, Donna Summer, Tina Charles, Gloria Gaynor and the Bee Gees. Rhythmic oldies stations still exist today, but target African-Americans as opposed to a mass audience."
}

In [ ]:
qg = main.QGen()

In [ ]:
output = qg.predict_shortq(payload)

In [ ]:
from pprint import pprint

In [ ]:
dataset_squad = datasets.load_dataset("squad_v2")

In [ ]:
qa_dict = {'what':[] , 'where': [], 'how':[], 'why':[], 'when':[], 'which':[],  'misc':[], 'who' : []}
qa_keys = ['what', 'where', 'how', 'why', 'when', 'which', 'who']
count = 0

## Generate Questions

In [ ]:
contexts = set()
count = 0
qa_dict = {'what':[] , 'where': [], 'how':[], 'why':[], 'when':[], 'which':[],  'misc':[], 'who' : []}
count = 0
for data in dataset_squad['train']:
  count += 1
  if count > 100:
    break
  if data['context'] not in contexts:
    contexts.add(data['context'])
    payload['input_text'] = data['context']
    output = qg.predict_mcq(payload)
    dataset_entry  = {'answers':{'answer_start':[],'text':[]},'context':data['context'],'id':None,'question':None,'title':data['title']}
    if 'questions' not in output:
      continue
    for question in output['questions']:
      dataset_entry['id'] = str(count)
      count +=1
      dataset_entry['question'] = question['question_statement']
      dataset_entry['answers']['text'] = [question['answer']]
      dataset_entry['answers']['answer_start'] = [data['context'].lower().find(question['answer'])]
      if dataset_entry['answers']['answer_start'][0] == -1:
        continue
      for key in qa_keys:
        misc = True
        if key in dataset_entry['question'].lower():
          qa_dict[key].append(dataset_entry)
          misc = False 
          break
        if misc == True:
          qa_dict['misc'].append(dataset_entry)

In [ ]:
import pandas as pd

In [ ]:
squad_train_df = pd.DataFrame(dataset_squad['train'])

In [ ]:
rows = []

for i in qa_dict:
  for j in qa_dict[i]:
    rows.append(j)

In [ ]:
temp = pd.DataFrame(rows)

In [ ]:
squad_train_df = squad_train_df.append(temp)

In [ ]:
import numpy as np

In [ ]:
squad_train_df = squad_train_df.drop('id', axis = 1)
squad_train_df['id'] = np.arange(len(squad_train_df))

In [ ]:
squad_train_df

In [ ]:
dataset_squad['train'] = datasets.Dataset.from_pandas(squad_train_df)

In [ ]:
dataset_squad

## Save to file

In [ ]:
import pickle as pkl
with open('/content/drive/MyDrive/Spring22/CS769/Project/FInal/SQUAD_gen.pkl' , 'wb') as fp:
  pkl.dump(dataset_squad, fp)